<a href="https://colab.research.google.com/github/OliviaIntan/Machine-Learning/blob/main/Tugas_Partisipatif_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Olivia Intan Puspita

NIM: 2311110006

Kelas: SD-04-01

In [1]:
# Import Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

## a. Memanggil Data

In [2]:
df = pd.read_excel('/content/CAD-Dataset.xlsx')

## b. Menampilkan data

In [3]:
df.head()

,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,1,53,90,175,Male,?,n,Y,Y,n,...,4.7,141,5700,39,52,261,50,0,N,Cad
1,2,67,70,157,Fmale,NaN,n,Y,n,n,...,4.7,156,7700,38,55,165,40,4,N,Cad
2,3,54,54,164,Male,NaN,n,n,Y,n,...,4.7,139,7400,38,60,230,40,2,mild,Cad
3,4,66,67,158,Fmale,NaN,n,Y,n,n,...,4.4,142,13000,18,72,?,55,0,Severe,Normal
4,5,50,87,153,Fmale,NaN,n,Y,n,n,...,4.0,140,9200,55,39,274,50,0,Severe,Normal


## c. Preprocessing data

Apakah ada data yang kosong? jika ada diapakan?

In [4]:
# Memeriksa data kosong
df.isnull().sum()

,0
No,0
Age,0
Weight,0
Length,0
Sex,0
BMI,302
DM,1
HTN,0
Current Smoker,4
EX-Smoker,0


### Missing value kolom BMI

Terlihat bahwa kolom ```BMI``` memiliki 302 missing values (hampir semua). Karena ```BMI``` penting dalam analisis kondisi pasien CAD, maka kita hitung ulang BMI dari kolom Weight dan Length untuk mendapat nilai pada kolom ```BMI```.

In [5]:
# Pastikan tidak ada nilai nol atau negatif di Weight dan Length
df = df[(df['Weight'] > 0) & (df['Length'] > 0)]

# Hitung ulang BMI
df['BMI'] = df['Weight'] / ((df['Length'] / 100) ** 2)

# Isi nilai NaN yang mungkin muncul akibat pembagian nol dengan median BMI
df['BMI'] = df['BMI'].fillna(df['BMI'].median())

# Verifikasi hasil
print(df[['Weight', 'Length', 'BMI']].head())


   Weight  Length        BMI
0      90     175  29.387755
1      70     157  28.398718
2      54     164  20.077335
3      67     158  26.838648
4      87     153  37.165193


### Missing value kolom numerik

Terlihat bahwa, kolom ```FBS``` dan kolom ```ESR``` memiliki missing value masing-masing 2 dan 1. Karena hanya sedikit missing value, kemungkinan data tersebut outlier, sehingga pengatasiannya menggunakan median.

In [6]:
# Tangani kolom numerik dengan median
num_cols = ['FBS', 'ESR']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

### Missing value kolom kategorikal

Terlihat bahwa kolom ```DM```, ```Current Smoker```, ```Nonanginal```, dan ```LowTH Ang``` memiliki missing value masing-masing 1, 4, 1, dan 2. Karena hanya sedikit missing value, dan kolom kategori tidak bisa dihitung rata-rata, sehingga pengatasiannya menggunakan mode atau nilai yang sering muncul.

In [7]:
# Tangani kolom kategorikal dengan mode
cat_cols = ['DM', 'Current Smoker', 'Nonanginal', 'LowTH Ang']
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verifikasi hasil
df.isnull().sum()

,0
No,0
Age,0
Weight,0
Length,0
Sex,0
BMI,0
DM,0
HTN,0
Current Smoker,0
EX-Smoker,0


## d. Mengubah semua kolom menjadi bernilai numerik

In [8]:
# Memeriksa tipe data semua kolom
df.dtypes

,0
No,int64
Age,int64
Weight,int64
Length,int64
Sex,object
BMI,float64
DM,object
HTN,object
Current Smoker,object
EX-Smoker,object


In [9]:
# Mengecek tipe data setiap kolom kategorikal
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols].dtypes

,0
Sex,object
DM,object
HTN,object
Current Smoker,object
EX-Smoker,object
FH,object
Obesity,object
CRF,object
CVA,object
Airway disease,object


In [10]:
# Mengkonversi semua data di kolom kategorikal menjadi string sebelum encoding
df[categorical_cols] = df[categorical_cols].astype(str)

# Melakukan label encoding
label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [11]:
# Verifikasi hasil
df.dtypes

,0
No,int64
Age,int64
Weight,int64
Length,int64
Sex,int64
BMI,float64
DM,int64
HTN,int64
Current Smoker,int64
EX-Smoker,int64


## e.	Melakukan proses normalisasi pada masing-masing atribut

Karena dataset ini berisi kondisi pasien Coronary Artery Disease (CAD), digunakan ```Min-Max Scaling``` dari ```sklearn.preprocessing.MinMaxScaler``` untuk menstandarkan semua fitur ke dalam rentang 0 hingga 1. ```Min-Max Scaling``` dipilih karena menjaga distribusi asli data dan memastikan setiap fitur berada dalam skala yang seragam, yang dapat meningkatkan performa model dalam analisis medis.

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Pilih hanya kolom numerik untuk normalisasi
numerical_cols = df.select_dtypes(include=['number']).columns

# Lakukan normalisasi
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Tampilkan hasil normalisasi
df.head()


,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,0.000000,0.410714,0.583333,0.729167,1.0,0.494721,1.0,0.0,0.333333,0.5,...,0.472222,0.464286,0.139860,0.603774,0.350877,0.686567,0.777778,0.0,0.333333,0.0
1,0.003311,0.660714,0.305556,0.354167,0.0,0.451314,1.0,0.0,1.000000,0.5,...,0.472222,1.000000,0.279720,0.584906,0.403509,0.074627,0.555556,1.0,0.333333,0.0
2,0.006623,0.428571,0.083333,0.500000,1.0,0.086105,1.0,1.0,0.333333,0.5,...,0.472222,0.392857,0.258741,0.584906,0.491228,0.500000,0.555556,0.5,1.000000,0.0
3,0.009934,0.642857,0.263889,0.375000,0.0,0.382846,1.0,0.0,1.000000,0.5,...,0.388889,0.500000,0.650350,0.207547,0.701754,1.000000,0.888889,0.0,0.666667,1.0
4,0.013245,0.357143,0.541667,0.270833,0.0,0.836058,1.0,0.0,1.000000,0.5,...,0.277778,0.428571,0.384615,0.905660,0.122807,0.761194,0.777778,0.0,0.666667,1.0
